## **Excel to CSV Aggregation**

This notebook preprocesses crop statistics data from the **State Statistics Service of Ukraine (SSSU)** for the years **2013 to 2023** and combines them into a single CSV file.

In [ ]:
import os

import numpy as np
import pandas as pd
import yaml

In [5]:
# load mapping dictionary from yaml file
with open('mapping.yaml', 'r', encoding='utf-8') as file:
    mapping = yaml.safe_load(file)

region_dict = mapping['region']
crop_dict = mapping['crop']
indicator_dict = mapping['indicator']

In [ ]:
base_path = "/home/scsi/Desktop/Sehoon/crop_yield/data/yield"

# List of .xlsx files
file_name_array = [file for file in os.listdir(base_path) if file.endswith('.xlsx')]
df_combined = pd.DataFrame()  # Initialize an empty DataFrame to store final data

for file_name in file_name_array:
    file_path = os.path.join(base_path, file_name)
    year = file_name.split('.')[0].split('_')[-1]  # Extract year from file name

    # Read Excel file, assuming a multi-index header
    data = pd.read_excel(file_path, header=[0, 1])

    # Set the first column (region name) as the index
    data.set_index(data.columns[0], inplace=True)

    # Convert data to long format
    df_long = data.stack(level=[0, 1], future_stack=True).reset_index()

    # Insert 'year' column
    df_long.insert(1, 'year', year)
    # Rename columns
    df_long.columns = ['region', 'year', 'crop_type', 'indicator', 'value']

    # Map region, crop, and indicator names to their respective IDs
    df_long['region_id'] = df_long['region'].map(region_dict).apply(lambda x: x[0])
    df_long['region_name'] = df_long['region'].map(region_dict).apply(lambda x: x[1])
    df_long['crop_id'] = df_long['crop_type'].map(crop_dict)
    df_long['indicator'] = df_long['indicator'].map(indicator_dict)

    # Reorder columns
    df_long = df_long[['region_id', 'region_name', 'year', 'crop_id', 'indicator', 'value']]

    # Concatenate data from each file
    df_combined = pd.concat([df_combined, df_long], ignore_index=True)

# Replace special characters and convert to numeric values
df_combined['value'] = df_combined['value'].replace(['...', '–'], np.nan)
df_combined['value'] = df_combined['value'].replace('к', -1).fillna(0)
df_combined['value'] = pd.to_numeric(df_combined['value'], errors='coerce')
df_combined['year'] = pd.to_numeric(df_combined['year'], errors='coerce')

# Use pivot table to reshape data
pivot_df = df_combined.pivot_table(
    index=['region_id', 'region_name', 'year', 'crop_id'],
    columns='indicator',
    values='value'
).reset_index()

print(pivot_df.dtypes)

# Save DataFrame to CSV file
pivot_df.to_csv('crop_yield.csv', index=False)